# Funciones de agregación y transformación

## Importamos un dataset de Kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rohitsahoo/sales-forecasting")

print("Path to dataset files:", path)

In [ ]:
!ls /kaggle/input/sales-forecasting

In [ ]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ahora copiar el dataset descargado a nuestro Google Drive
import os, shutil
# Carpeta destino
dest_dir = "/content/drive/MyDrive/datasets/"

# Copiar todos los archivos del dataset al destino
for file in os.listdir(path):
    shutil.copy(os.path.join(path, file), dest_dir)

# print("Archivos copiados a:", dest_dir)
# print("Contenido:", os.listdir(dest_dir))

os.listdir(dest_dir)

In [ ]:
import pandas as pd

In [ ]:
# Usar el método read_csv de Pandas (pd) que recibe como argumento el archivo csv y retorna un dataframe
df = pd.read_csv("/content/drive/MyDrive/datasets/train.csv")
df.head()

## Drop de columnas que no nos interesan


In [ ]:
df.drop(columns=["Order ID", "Order Date", "Customer ID", "Postal Code", "Product ID"], inplace=True)
df.head()

## Filtros y Transformaciones - Parte B

### Filtros convencionales

In [ ]:
# Lista solo los registros del campo State. Probar como cambia con unique
df["State"].unique()

In [ ]:
# Filtrar los registros de un estado específico, por ejemplo Indiana, Illinois, Wisconsin
df[df["State"]=="Indiana"]

In [ ]:
# Filtrar los registros de un estado específico y mejorar con transformación
df[df["State"].str.lower()=="indiana"]

In [ ]:
# Filtrar los registros de un estado usando contains
df[df["State"].str.contains("indi", case=False)]

In [ ]:
# Filtrar los registros de una lista de estados con isin()
df[df["State"].isin(["Indiana", "California"])].sort_values("State", ascending=True)

In [ ]:
# Filtrar los registros de una lista de estados con isin() y mejorar con trasnformaciones
df[df["State"].str.lower().isin(["indiana", "california"])].sort_values("State", ascending=True)

### Filtros con filter

`filter()` no usa condiciones lógicas sobre los datos (para eso se usa query(), loc[], o comparaciones booleanas).
Solo se basa en los nombres de columnas o índices.

Es muy útil para seleccionar columnas por patrón en datasets grandes.

Se puede combinar con otros métodos, por ejemplo:

In [ ]:
# Filtrar columnas por nombre exacto
df.filter(items=['Country', 'State'])

In [ ]:
# Filtrar columnas que contienen una cadena de caracteres
df.filter(like='ountry', axis=1)

In [ ]:
# Filtrar columnas con expresión regular
df.filter(regex=r'^S|s', axis=1)

In [ ]:
# Filtrar filas (usando índices)
df.filter(like='2024', axis=0)

### Filter con query

In [ ]:
# Seleecionar las ventas mayores a 1000
df.query("Sales > 1000")

In [ ]:
# Seleecionar las ventas mayores a 1000 y menores a 1500
df.query("Sales > 1000 & Sales < 1500")

In [ ]:
# Seleecionar las ventas mayores a 1000 del estado de Illinois
df.query("Sales > 1000 & State == 'Illinois'")

In [ ]:
# Seleccionar los registros cuyo campo State contenga california (ignorar case)
df.query("State.str.contains('california', case=False, na=False)")

## Agregación

Las posibles agreaciones son:
* count() cuenta los valores no nulos
* sum() suma todos los valores numericos
* mean() calcula la media aritmética
* min(), max() retorna el valor más bajo, más alto
* std() retorna la medida de dispersión

In [ ]:
df.info()

In [ ]:
# Obtener las suma de las ventas registradas por categoría
df.groupby(["Category"], as_index=False)["Sales"].sum()

In [ ]:
# Obtener la suma de las ventas registradas por estado
df.groupby(["State"], as_index=False)["Sales"].sum()

In [ ]:
# Obtener el promedio de las ventas promedio por segmento
df.groupby(["Segment"], as_index=False)["Sales"].mean()

In [ ]:
# Calular la suma de las ventas, agrupadas por categoría y por sub-categoría
df.groupby(["Category", "Sub-Category"])["Sales"].sum()

In [ ]:
# Repetir la consulta anterior usando el método agg
df.groupby(["Category", "Sub-Category"], as_index=False).agg({"Sales": "sum"})

In [ ]:
# Repetir la consulta anterior y agregar además la cantidad de la Sub-Category
df.groupby(["Category", "Sub-Category"], as_index=False).agg({"Sales": "sum", "Sub-Category": "count"})

In [ ]:
# Agrupando por Category y Sub-Category, generar 3 columnas con:
# Suma de Sales como total_sales
# Cantidad de Sales como count_sales
# Promedio de Sales como avg_sales
# Validar el promedio
df.groupby(["Category", "Sub-Category"], as_index=False).agg(
    total_sales=("Sales", "sum"),
    count_sales=("Sales", "count"),
    avg_sales=("Sales", "mean")
)

In [ ]:
# A la comparacion de ventas por Categoria y Subcategoria con categoria total, agregar una columna extra con el indice
agg_df = df.groupby(["Category", "Sub-Category"]).agg(sug_cat_sales=("Sales", "sum"))
agg_df["cat_sales"] = agg_df.groupby(["Category"])["sug_cat_sales"].transform("sum")
agg_df["ratio_%"]= (agg_df["sug_cat_sales"] / agg_df["cat_sales"]*100).round(2)
agg_df

## Agregaciones y transformaciones avanzadas

In [ ]:
# Analizar que porcentaje representa cada subcategoría de su máximo
agg_df = (
    df.groupby(["Category", "Sub-Category"], as_index=False)["Sales"]
      .sum()
)

# Calculamos el máximo dentro de cada categoría
agg_df["max_sales_cat"] = agg_df.groupby("Category")["Sales"].transform("max")

# # Comparamos con ese máximo
agg_df["porcentaje_vs_max"] = agg_df["Sales"] / agg_df["max_sales_cat"] * 100
agg_df